In [2]:
from konlpy.tag import Okt, Kkma

In [3]:
okt=Okt()
kkma = Kkma()

SystemError: java.nio.file.InvalidPathException: Illegal char <*> at index 65: C:\Users\smhrd\anaconda3\envs\deep\lib\site-packages\konlpy\java\*

### CounterVectorizer와 연결해서 사용하기

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
countVec = CountVectorizer()

In [ ]:
text = ["알고리즘 트레이딩의 전략은 무수히 많을 것이다. 아래 그림은 이미 알려진 몇 가지 전략들을 (두서없이) 나열해본 것이다. 여기서는 시장의 미시적 측면에서 LOB의 균형관계를 이용한 전략에 대해 (앞으로) 하나씩 알아보기로 한다. 미시 전략에서는 주가나 변동성 자체를 분석하지는 않는다. 그 이유는 미시 전략은 대단히 빠른 시간 안에 이루어지고, 주가는 미시시장의 최종 결과로 발생하는 것이기 때문에 주가를 보면 이미 늦은 감이 있다고 볼 수 있기 때문이다. 따라서 미시 전략은 주가 발생의 원인을 분석한다. 즉, LOB에서 발생하는 사건의 흐름 (잔량의 변화나 주문의 흐름)을 분석한다 "]

In [ ]:
countVec.fit(text) #토큰화 및 단어사전 구축

In [ ]:
countVec.vocabulary_ #기본 띄어쓰기 단위로 토큰화

In [ ]:
def myTokenizer(text): #나만의 토큰화 도구 만들기
    return okt.nouns(text) #명사만 추출해서 토큰화하는 도구
                            # 필요에 따라서 pos를 활용해서 원하는 부분만 추출하도록 추가

In [ ]:
countVecWithKonlpy = CountVectorizer(tokenizer=myTokenizer)

In [ ]:
countVecWithKonlpy.fit(text)

In [ ]:
countVecWithKonlpy.vocabulary_

### 네이버 영화리뷰 데이터셋 감성분석

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
# 데이터 로딩 & 결측치 삭제

In [ ]:
text_train = pd.read_csv("data/ratings_train.txt", delimiter ="\t")

In [ ]:
text_test = pd.read_csv("data/ratings_test.txt", delimiter ="\t")

In [ ]:
text_train.dropna(inplace=True)
text_test.dropna(inplace=True)

In [ ]:
text_train.shape, text_test.shape

In [ ]:
# 문제와 정답 나누기
X_train = text_train["document"]
y_train = text_train["label"]
X_test = text_test["document"]
y_test = text_test["label"]

#### 감성분석 실습
- 1. pipeline 모델 구축(CountVectorizer, LogisticRegression)
- 2. 모델 학습(train)
- 3. 모델 평가(test)
- 4. 긍/부정 리뷰에 영향을 많이 주는 한국어 단어를 시각화

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

In [ ]:
pipe_model = make_pipeline(CountVectorizer(), LogisticRegression())

In [ ]:
pipe_model.fit(X_train, y_train) # 토큰화 + 수치화 -> 모델 학습

In [ ]:
pipe_model.score(X_test, y_test)

In [ ]:
final_cv=pipe_model.steps[0][1]

In [ ]:
len(final_cv.vocabulary_)

In [ ]:
final_cv.vocabulary_

In [ ]:
final_logi = pipe_model.steps[1][1]

In [ ]:
word_weights = final_logi.coef_
word_weights

In [ ]:
word_weights.shape

In [ ]:
df = pd.DataFrame([final_cv.vocabulary_.keys(), final_cv.vocabulary_.values()])
df

In [ ]:
df = df.T # 1번 열을 기준으로 정렬
df

In [ ]:
df_sorted = df.sort_values(by=1) # 1번 열을 기준으로 정렬
df_sorted

In [ ]:
df_sorted["coef"] = word_weights.reshape(-1) # 가중치를 1차원으로 변경후 컬럼에 추가
df_sorted

In [ ]:
df_sorted.sort_values(by="coef", inplace=True) #coef 기준으로 정렬
df_sorted

In [ ]:
top30_df = pd.concat([df_sorted.head(30), df_sorted.tail(30)])
top30_df

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager,rc
font_name = font_manager.FontProperties(fname='C:\Windows\Fonts\malgun.ttf').get_name()
rc('font',family=font_name)

In [ ]:
plt.figure(figsize=(15,5)) #가로 세로 비율
plt.bar(top30_df[0],top30_df["coef"]) # x축은 단어, y축은 가중치
plt.xticks(rotation=90) # x축 눈금 각도 조정
plt.show()